In [1]:
import pandas as pd
import pprint
import numpy as np
import sklearn.preprocessing as preprocessing
from IPython.core.debugger import Tracer
import json
import operator
from collections import OrderedDict
import collections

# TODO: Temos um erro no modelo, que é usar o labelEncoder na coluna de segmento, se estivermos
# implementando um modelo de ML, isso seria um problema, o modelo 'pensaria' que as categorias com
# maior valor são mais importantes, quando na verdade, têm a mesma importância [FIXED]

# Como estamos usando uma lógica própria e não um algoritmo de ML, esse problema não impacta
# A comparação que fazemos é se o segmento do usuario a receber a recomendação é igual ao segmento
# do usuário que poderá ser recomendado

In [91]:
dados_json = pd.read_json("dados.json")

dados_json['user_id'] = range(0,len(dados_json))
dados_json = dados_json.reindex_axis(['user_id','tipo_user','habilidades','segmento','horas_disponiveis'], axis=1) #reordenamos somente pra melhorar a visualização e ficar igual csv
#new_dados =  dados_json.to_csv(path_or_buf="./new_dados.csv",date_format="utf-8", index=False)
dados_json.head()

,user_id,tipo_user,habilidades,segmento,horas_disponiveis
0,0,startup,"[python, php, js, scrum]",fintech,6
1,1,user,"[python, js, illustrator]",edtech,8
2,2,startup,"[laravel, php, js, html]",biotech,4
3,3,startup,"[scrum, laravel, angular, html]",adtech,6
4,4,startup,"[js, html, css]",adtech,8


In [92]:
## Preprocessing data here:

# Transformando coluna tipo_user
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(dados_json['tipo_user'])
label_encoder.classes_
dados_json['tipo_user'] = label_encoder.transform(dados_json['tipo_user'])

# Retirando linhas com habilidades em branco
for line in dados_json.itertuples():
    if line.habilidades == ['']:
#         print (line)
        dados_json = dados_json[dados_json.user_id != line.user_id]
#         print(50*"*")

# Criando array com habilidades para usar no dict de pesos
_habilidades = []
for habilidades in dados_json['habilidades']:
    for habilidade in habilidades:
        _habilidades.append(habilidade)
_habilidades = set(_habilidades)

# Arrumando os indexes do df após deletar linhas com habilidades em branco
# Fizemos isso por causa do .join, que estava dando problema de index e gerando NaN
dados_json = dados_json.reset_index(drop=True)

mlb = preprocessing.MultiLabelBinarizer()
habilidades_col = mlb.fit_transform(dados_json['habilidades'])
 
habilidades_df = pd.DataFrame(habilidades_col,columns=mlb.classes_)
dados_json = dados_json.join(habilidades_df)

# Deletamos a coluna categorica de habilidades
dados_json.drop('habilidades',axis=1,inplace=True)
dados_json.head()


,user_id,tipo_user,segmento,horas_disponiveis,angular,big-data,branding,cakephp,css,gestao-projetos,...,machine-learning,marketing-digital,photoshop,php,python,react,scikit-learn,scrum,scss,seo
0,0,0,fintech,6,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,0,0
1,1,1,edtech,8,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,2,0,biotech,4,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,3,0,adtech,6,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,4,0,adtech,8,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
len(_habilidades)

21

In [94]:
dados_json.shape

(119, 25)

In [93]:
#Conferindo formato csv
dados_csv = pd.read_csv("dados.csv",sep=";") 
dados_csv.head()

,tipo_user,habilidades,segmento,horas_disponiveis
0,startup,"python,php,js,scrum",fintech,6
1,user,"python,js,illustrator",edtech,8
2,startup,"laravel, php, js, html",biotech,4
3,startup,"scrum,laravel,angular,html",adtech,6
4,startup,"js,html,css",adtech,8


In [7]:
dados_json.columns

Index(['user_id', 'tipo_user', 'segmento', 'horas_disponiveis', 'angular',
       'big-data', 'branding', 'cakephp', 'css', 'gestao-projetos', 'html',
       'illustrator', 'ionic', 'js', 'laravel', 'machine-learning',
       'marketing-digital', 'photoshop', 'php', 'python', 'react',
       'scikit-learn', 'scrum', 'scss', 'seo'],
      dtype='object')

In [ ]:
class Ranking (object):

    def __init__(self,df,preprocessed=True):
        self.df = df
        
    def getDf (self):
        return self.df
    
    def preprocessDataFrame (self): # testar se é preciso receber df como parametro
        categorical_columns = self.df.select_dtypes(['object']).columns
        label_encoder = preprocessing.LabelEncoder()
        for column in categorical_columns:
            label_encoder.fit(self.df[column])
            label_encoder.classes_
            self.df[column] = label_encoder.transform(self.df[column])
    
    def train_test_split (self,test_size=0.6):
        limit = int((len(self.df)*test_size))
        X_treino = self.df[:limit]
        X_teste = self.df[limit:].reset_index(drop=True)
        return X_treino, X_teste

    '''
        Função que gera um dicionario de rankings dado um conjunto de teste(à ser recomendado) 
        e um conjunto de treinamento para realizar a recomendação
    '''
    # Do jeito que rank_the_data está hoje, estamos sujeitos à ter que fazer manutenção constante no código
    # Somente na parte dos pesos, talvez tornamos ele um dicionario cuja chave seja o nome da coluna com seu valor
    # o peso respectivo => TODO <=
    def rank_the_data(self,X_treino,X_teste,pesos,name_treino='Ranking_Treino', name_teste='Ranking_Teste'):

        temp_matches = {}
        matches_dict = {}

        for row in X_teste.itertuples(name=name_teste): # definindo a row atual do x_teste
            match = False
            valores_teste  = []
            valores_treino = []
            rating = 0

#             pesos  = (len(row) - 3) * [0.0] #array com os pesos
            # o -3 representa as três colunas que não precisamos (index,user_id,user_type)
#             pesos[0] = 0.7 # aumentando peso do segmento para quando houver match

            
#             for i in range(2,len(pesos)):
#                 pesos[i] = 1.0 # aumentando o peso das habilidades
            # para a disponibilidade de horas nós temos que fazer um calculo mais detalhado
            # diponibilidade_startup > disponibilidade_user; disponibilidade_user/disponibilidade_startup => peso
            # diponibilidade_startup < disponibilidade_user; disponibilidade_startup/disponibilidade_user => peso

            for i in range(3,len(row)):  
            # aqui é o loop do item da linha (colunas) começamos do index 2 por causa da estrutura da tupla
#                 print("row[i]: ", row[i])
                valores_teste.append(row[i])

            for row_treino in X_treino.itertuples(name=name_treino):# loop da row atual do x_treino
                

                if not(row.tipo_user == row_treino.tipo_user): # nao podemos computar valores para usuarios de mesmo tipo
                    # para adicionar funcionalidade de peso teremos que modificar a partir da current_sum
                    # pensamento: e se já de inicio nós tivermos um array com todos os pesos settados com 1 e fizermos
                    # a multiplicação
                    for i_treino in range(3, len(row_treino)): # loop dos itens da linha (colunas)
                    valores_treino.append(row_treino[i_treino]) # valor da coluna
                    #peso disp_horas
                    # se a qtd de horas que buscamos / qtd de horas que o user tem
                    # for menor que 1, temos que atualizar o valor do peso com a 
                    # penalidade

  
                    if row_treino.horas_disponiveis / row.horas_disponiveis < 1:
                        pesos[1] = pesos[1] * (row_treino.horas_disponiveis/row.horas_disponiveis)
                     
                    if row_treino.horas_disponiveis == row.horas_disponiveis:
                        current_sum = (sum(np.equal(valores_teste, valores_treino)*pesos))/(len(X_teste.columns) - 2)
                    else:
                        current_sum = (sum(np.equal(valores_teste, valores_treino)*pesos)+ pesos[1])/(len(X_teste.columns) - 2)
                    rating = np.around(current_sum, decimals=2)
                    temp_matches[row_treino] = float(format(rating, '.2f'))
                valores_treino = [] #resetamos o valor a cada iteração que a row de treinamento muda
                Tracer()() 
            matches_dict[row] = temp_matches
            temp_matches = {}
            
        return matches_dict
    
    def filter_ranked_data (self,matches_dict, min_val=0.65):
        results = {}
        for key,value in matches_dict.items():
            filtered = {k:v for k,v in value.items() if v >= min_val}
            
            for k,v in filtered.items():
                results[key.user_id] = {k.user_id:v for k,v in filtered.items()}

        return results #retornamos um dicionario com os dados filtrados, isso é o que temos que mandar através da api e tratar
    
    def visualize_recommendations(self,filtered_dict, categoric_df):
    
        for key, value in filtered_dict.items():
            print("KEY: ", key)
            test_data = categoric_df.loc[categoric_df["user_id"] == key]
            columns = categoric_df.columns
            print("-----------------------")
            print("Dado para teste: ")
            print("-----------------------")

            for i in range(len(columns)):
                print("{0}: {1}".format(columns[i], test_data.loc[:,[columns[i]]].values[0]))
#             print(test_data)
            print("---------------------------------------------")
            print("Recomendações: ")
            print("-----------------------")

            value = collections.OrderedDict(sorted(value.items(),key=lambda x: x[1],reverse=True)) #ordenando por valores(ratings)
            
            for k,v in value.items():
                rating = v
                rating_user = categoric_df.loc[categoric_df["user_id"] == k,:]
                print("Recomendação: %.2f%%" % (rating*100))
                for i in range(len(columns)):
                    print("{0}: {1}".format(columns[i], rating_user.loc[:,[columns[i]]].values[0]))
                print("----")

In [ ]:
def generateWeights (X,weights_dict)  :
    size = len(X.columns) - 2 # the subtraction represents the first two columns we don't want
    weights = size * [0.0]
    weights[0]= weights_dict["segmento"]
    weights[1]= weights_dict["horas_disponiveis"]
    weights[2:] = len(weights[2:])*[weights_dict["habilidades"]]
    return weights

# Generate the weights
pesos = OrderedDict({"segmento":0.7,"horas_disponiveis":0.75, "habilidades":1.0})
X_treino  = dados_json[:len(dados_json)-2] 
X_teste   = dados_json[len(dados_json)-2:]
weights = generateWeights(X_treino,pesos)

In [ ]:
dados_raw = pd.read_csv("new_dados.csv")
rank      = Ranking(dados_json)
rank.preprocessDataFrame()

ranked_data = rank.rank_the_data(X_treino, X_teste,weights)
print(ranked_data)
filtered_data = rank.filter_ranked_data(ranked_data,min_val=0.80) #retorna a % e o id do usuario relacionado
print("*********")
print(filtered_data)

In [140]:
rank.visualize_recommendations(filtered_data, dados_raw)

KEY:  136
-----------------------
Dado para teste: 
-----------------------
user_id: [136]
tipo_user: ['startup']
habilidades: ["['machine-learning', 'php', 'cakephp']"]
segmento: ['adtech']
horas_disponiveis: [8]
---------------------------------------------
Recomendações: 
-----------------------
Recomendação: 83.00%
user_id: [94]
tipo_user: ['user']
habilidades: ["['cakephp']"]
segmento: ['biotech']
horas_disponiveis: [4]
----
Recomendação: 82.00%
user_id: [15]
tipo_user: ['user']
habilidades: ["['machine-learning', 'ionic', 'js', 'php']"]
segmento: ['adtech']
horas_disponiveis: [6]
----
Recomendação: 81.00%
user_id: [59]
tipo_user: ['user']
habilidades: ["['machine-learning', 'angular']"]
segmento: ['adtech']
horas_disponiveis: [8]
----


In [ ]:
def generateFullJSON(raw_df,filtered_dict):
    #the JSON should return the following structure:
    jsons      = []
    for key,item in filtered_dict.items():
        user_type = raw_df.loc[:,'tipo_user'][key]
        user_id = key
        skills = raw_df.loc[:,'habilidades'][key]
        market = raw_df.loc[:, 'segmento'][key]
        hours = raw_df.loc[:, 'horas_disponiveis'][key]
        codigo = collections.OrderedDict({"user_id": int(user_id), "user_type": user_type, 
                  "skills":skills,"market":market,"hours":float(hours),"ratings": []})
        
        #ordenando pela rating
        ordered_dict = collections.OrderedDict(sorted(item.items(), key= lambda x:x[1],reverse=True))
        #print("Ordered dict", ordered_dict)
        for k,v in ordered_dict.items():
            
            guess_user_type = raw_df.loc[:, 'tipo_user'][k]
            guess_user_id = k
            guess_skills = raw_df.loc[:,'habilidades'][k]
            guess_market = raw_df.loc[:, 'segmento'][k]
            guess_hours = raw_df.loc[:, 'horas_disponiveis'][k]
            
            codigo["ratings"].append(collections.OrderedDict({"user_id": int(guess_user_id), "user_type": guess_user_type, 
                  "skills":guess_skills,"market":guess_market,"hours":float(guess_hours),"rating":float(v)}))
            
        jsons.append(codigo)
    return json.dumps(jsons)
            
        
    

In [ ]:
JSON = generateFullJSON(dados_raw, filtered_data)
JSON

In [ ]:
pesos ={}

for column in X_treino.columns[2:]:
    pesos[column] = 0
pesos

In [ ]:
for peso in pesos :
    if peso not in ['segmento','horas_disponiveis']:
        print(peso)

In [ ]:
a = 0.9400000000000
a = float(format(a, '.2f'))

In [ ]:
type(a)

In [ ]:
pesos = OrderedDict({"php":1.0, "python":1.0,"segmento":0.6})
list(pesos.values())

In [123]:
test = dados_raw.loc[dados_raw["user_id"] == 136]

In [136]:
print(test.loc[:,['segmento']].values[0])

['adtech']


In [ ]:
len(X_treino.columns) - 2